# Non Adaptive Importance Sampling algorithm with openturns

Source of the algorithm : J. Morio & M. Balesdent, Estimation of Rare Event Probabilities in Complex Aerospace and Other Systems, A Practical Approach, Elsevier, 2015


The theory is given for a failure event defined as $\phi(\mathbf{X})>S$ with $\mathbf{X}$ a random vector following a joint PDF $h_0$, $S$ a threshold and $\phi$ a limit state function, without loss of generality.

The IS probability estimate by Importance Sampling $\widehat{P}^{IS}$ is given by 
\begin{equation}
\widehat{P}^{IS}=\frac{1}{N} \sum_{i=1}^{N} {\bf 1}_{\phi(\mathbf{X}_i)>S} \frac{h_0(\mathbf{X}_i)}{h(\mathbf{X}_i)}.
\label{ISeq}
\end{equation}

It is well-known that the optimal density minimizing the variance of the estimator $h_{opt}$ is  defined as
\begin{equation}
h_{opt}=\frac{{\mathbf 1}_{\phi(x)>S}h_0}{P}.
\label{opt}
\end{equation}


with $P$ the failure probability and is inaccessible in practice since this probability is unknown. 


The objective of Non parametric Adaptive Importance Sampling (NAIS) technique is to approximate the IS optimal auxiliary density given in the preceding equation  with a kernel function (e.g. Gaussian kernel). NAIS does not require the choice of a parametric pdf family  as Cross Entropy and is thus more flexible than a parametric model. Its iterative principle is relatively similar to CE optimization and can be described by the following steps.


1. $k=1$ and set $\rho \in [0,1]$
2. Generate the population $\mathbf{X}_1^{(k)},\dots,\mathbf{X}_N^{(k)}$ according to the pdf $h_{k-1}$, apply the function $\phi$ in order to have $Y_1^{(k)}=\phi(\mathbf{X}_1^{(k)}),\ldots,Y_N^{(k)}=\phi(\mathbf{X}_N^{(k)})$
3. Compute the empirical $\rho$-quantile $q_k=\min(S, Y^{(k)}_{\left\lfloor\rho N\right\rfloor})$
4. Estimate $I_k= \frac{1}{kN} \displaystyle \sum_{j=1}^{k}\sum_{i=1}^{N} {\bf 1}_{\phi(\mathbf{X}_i^{(j)}) \geq q_k} \frac{h_0(\mathbf{X}_i^{(j)})}{h_{j-1}(\mathbf{X}_i^{(j)})}$ 
5. Update the Gaussian kernel sampling pdf with
\begin{equation}
h_{k}(\mathbf{X})=\frac{1}{k N I_k \det\left(B_{k+1}\right)}\sum_{j=1}^{k}\sum_{i=1}^{N}  w_{j}(\mathbf{X}_i^{(j)})K_d\left(B_{k+1}^{-1}\left(\mathbf{X}-\mathbf{X}_i^{(j)}\right)\right),
\end{equation}
where $K_d$ is the standard $d$-dimensional Gaussian function with zero mean and a diagonal covariance matrix $B_{k+1}=diag(b^1_{k+1},...,b^d_{k+1})$ and $w_j={\bf 1}_{\phi(\mathbf{X}_i^{(j)}) \geq q_k} \frac{h_0(\mathbf{X}_i^{(j)})}{h_{j-1}(\mathbf{X}_i^{(j)})}$. The coefficients of matrix $B_{k+1}$ can be approximated (Silverman Rule) or postulated according to the AMISE (Asymptotic Mean Integrated Square Error) criterion for example. 
6. If $q_k<S$, $k\leftarrow k+1$, go to Step 2
7. Estimate the probability $\widehat{P}^{NAIS}(\phi(\mathbf{\mathbf{X}}>S))=\frac{1}{N}\displaystyle \sum_{i=1}^{N} 1_{\phi(\mathbf{X}_i^{(k)})>S} \frac{h_0(\mathbf{X}_i^{(k)})}{h_{k-1}(\mathbf{X}_i^{(k)})}$


The NAIS algorithm with the Silverman rule is coded in the following class.


In [ ]:
import numpy as np
import openturns as ot
from NAISAlgorithm import NAISAlgorithm 

## Numerical experiments

http://openturns.github.io/openturns/master/examples/reliability_sensitivity/estimate_probability_monte_carlo.html


We consider a simple beam stressed by a traction load F at both sides.

The geometry is supposed to be deterministic; the diameter D is equal to:

$D=0.02 \textrm{ (m)}.$

By definition, the yield stress is the load divided by the surface. Since the surface is \pi D^2/4, the stress is:

$S = \frac{F}{\pi D^2/4}.$

Failure occurs when the beam plastifies, i.e. when the axial stress gets larger than the yield stress:

$R - \frac{F}{\pi D^2/4} \leq 0$

where R is the strength.

Therefore, the limit state function G is:

$G(R,F) = R - \frac{F}{\pi D^2/4},$

for any R,F\in\mathbb{R}.

The value of the parameter D is such that:

$D^2/4 = 10^{-4},$

which leads to the equation:

$G(R,F) = R - \frac{F}{10^{-4} \pi}.$

with

$R \sim LogNormal(\mu_R=3\times 10^6, \sigma_R=3\times 10^5) [Pa]$

$F \sim Normal(\mu_F=750, \sigma_F=50) [N]$

In [ ]:
#Creation of the event
distribution_R = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_F = ot.Normal(75e3, 5e3)
marginals = [distribution_R, distribution_F]
distribution = ot.ComposedDistribution(marginals)

# create the model
model = ot.SymbolicFunction(['R', 'F'], ['R-F/(pi_*100.0)'])

#create the event 
vect = ot.RandomVector(distribution)
G = ot.CompositeRandomVector(model, vect)
event = ot.ThresholdEvent(G, ot.Less(), 0.0)

In [ ]:
#Determination of reference probability
#MonteCarlo experiment
n_MC = 1e6

# create a Monte Carlo algorithm
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)
algo.setMaximumOuterSampling(int(n_MC))
algo.setMaximumCoefficientOfVariation(0.01)
algo.run()
# retrieve results
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)

In [ ]:
# Hyperparameters of the algorithm
n_IS= 2500 # Number of samples at each iteration
rho_quantile= 25 # Quantile determining the percentage of failure samples in the current population 

# Definition of the algoritm
NAIS = NAISAlgorithm(event,n_IS,rho_quantile)

# Run of the algorithm
NAIS.compute_proba()
print('Probability of failure:',NAIS.getFailureProbability())
